# Análisis de Dinámicas de Tráfico (Semana Aleatoria)

Este notebook analiza los patrones de tráfico de diferentes servicios (SMS, Llamadas, Internet). 
**Enfoque:** Se selecciona **una semana aleatoria** del conjunto de datos (de Lunes a Domingo) para analizar los patrones temporales.

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import random
import os

# Configuración de visualización
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 8)

## Carga y Procesamiento Temporal

Cargamos los datos utilizando Dask para eficiencia.

In [ ]:
files = ['data1.csv/data1.csv', 'data2.csv/data2.csv']
services = ['smsin', 'smsout', 'callin', 'callout', 'internet']

valid_files = [f for f in files if os.path.exists(f)]
if not valid_files:
    raise FileNotFoundError("No se encontraron los archivos de datos.")

print(f"Cargando archivos: {valid_files}")

# Leer CSVs con Dask
ddf = dd.read_csv(valid_files, assume_missing=True)

# --- AGREGACIÓN TEMPORAL ---
ddf_time = ddf[['TimeInterval'] + services]
agg_task = ddf_time.groupby('TimeInterval')[services].sum()

print("Procesando datos temporales...")
with ProgressBar():
    final_df = agg_task.compute()

# Post-procesamiento
final_df = final_df.reset_index()
final_df['Timestamp'] = pd.to_datetime(final_df['TimeInterval'], unit='ms')
final_df = final_df.sort_values('Timestamp')
final_df.set_index('Timestamp', inplace=True)

print("Rango de fechas total disponible:")
print(f"Inicio: {final_df.index.min()}")
print(f"Fin:    {final_df.index.max()}")

## Selección de Semana Aleatoria (Lunes - Domingo)

Seleccionamos una semana al azar dentro del rango de fechas disponible, asegurando que comience en **Lunes**.

In [ ]:
# Rango total de fechas
min_date = final_df.index.min()
max_date = final_df.index.max()

# Encontrar el primer lunes disponible
days_to_first_monday = (7 - min_date.weekday()) % 7
first_monday = min_date + pd.Timedelta(days=days_to_first_monday)
first_monday = first_monday.normalize()

# Calcular cuántas semanas completas hay
total_days = (max_date - first_monday).days
total_weeks = total_days // 7

if total_weeks < 1:
    print("Advertencia: Menos de una semana completa de datos desde el primer lunes. Usando la primera semana disponible ajustada.")
    start_date = first_monday
else:
    # Elegir una semana aleatoria
    random_week_offset = random.randint(0, total_weeks - 1)
    start_date = first_monday + pd.Timedelta(weeks=random_week_offset)

end_date = start_date + pd.Timedelta(days=7)

# Filtrar el DataFrame
week_df = final_df[(final_df.index >= start_date) & (final_df.index < end_date)]

print(f"\nAnalizando semana aleatoria: del {start_date.date()} (Lunes) al {end_date.date()} (Domingo)")
display(week_df.head())

## Visualización Semanal

Gráficas detalladas del comportamiento del tráfico durante la semana seleccionada.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

# Gráfico para Internet
ax1.plot(week_df.index, week_df['internet'], color='purple', label='Internet', linewidth=2)
ax1.set_title(f'Tráfico de Internet (Semana del {start_date.date()})', fontsize=14)
ax1.set_ylabel('Volumen de Datos')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Gráfico para SMS y Llamadas
colors = {'smsin': 'green', 'smsout': 'lightgreen', 'callin': 'blue', 'callout': 'cyan'}
for service in ['smsin', 'smsout', 'callin', 'callout']:
    ax2.plot(week_df.index, week_df[service], label=service, color=colors.get(service), alpha=0.8)

ax2.set_title(f'Tráfico de SMS y Llamadas (Semana del {start_date.date()})', fontsize=14)
ax2.set_xlabel('Tiempo')
ax2.set_ylabel('Volumen de Eventos')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Forzar límites del eje X
ax2.set_xlim(start_date, end_date)

# Formatear eje X
import matplotlib.dates as mdates
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%A %H:%M'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Distribución Espacial 3D

Visualización en 3D de la distribución del tráfico SMS en la cuadrícula urbana (100x100). La altura (eje Z) representa el volumen de tráfico.

In [ ]:
# --- AGREGACIÓN ESPACIAL ---
print("Procesando distribución espacial...")
ddf_spatial = ddf[['GridID'] + services]
spatial_task = ddf_spatial.groupby('GridID')[services].sum()

with ProgressBar():
    spatial_df = spatial_task.compute()

spatial_df['sms_total'] = spatial_df['smsin'] + spatial_df['smsout']

# --- CREACIÓN DE LA MATRIZ ---
grid_matrix = np.zeros((100, 100))

for grid_id, row in spatial_df.iterrows():
    if 1 <= grid_id <= 10000:
        r = int((grid_id - 1) // 100)
        c = int((grid_id - 1) % 100)
        grid_matrix[r, c] = row['sms_total']

# --- PLOT 3D ---
fig = plt.figure(figsize=(15, 12))
ax = fig.add_subplot(111, projection='3d')

# Crear mallas de coordenadas X e Y
x = np.arange(0, 100, 1)
y = np.arange(0, 100, 1)
X, Y = np.meshgrid(x, y)

# Graficar superficie
# Usamos cmap='inferno' para mantener la consistencia con la idea de "calor"
surf = ax.plot_surface(X, Y, grid_matrix, cmap='inferno', edgecolor='none', alpha=0.9)

ax.set_title('Spatial Distribution (3D)', fontsize=18)
ax.set_xlabel('Grid X')
ax.set_ylabel('Grid Y')
ax.set_zlabel('SMS Traffic Volume')

# Añadir barra de color
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='Volumen de Tráfico')

# Ajustar ángulo de visión para mejor perspectiva
ax.view_init(elev=30, azim=45)

plt.tight_layout()
plt.show()